<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-02 13:01:10
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20


-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: -13.96 K (-0.1%)
Current PnL: -13.65 L (-9.51%)
CY Booked + Current PnL: -6.91 L (-4.81%)
-------------------
Total profit:  4.57 L
Total loss:  -18.21 L
-------------------
Total Booked + Current PnL: 20.07 L (17.23%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.95%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 72.08 L (52.9%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.64%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,HINDALCO,651.95,761.55,-18.25,H-LC,90.62,112424.0,8112.0,9421.0,1.20,7.78,8.38,16.81,27.0,0.86,0.83,25.07,X5K,ATH,METALS
87,UJJIVANSFB,52.77,53.00,54.03,M-SC,89.58,133596.0,-8883.0,9499.0,0.53,-6.23,7.11,0.44,243.0,-0.94,0.98,57.68,OX40N,NTT,BANKS
88,UNIONBANK,123.87,163.00,0.15,M-LC,39.58,175826.0,34986.0,9512.0,0.68,24.84,5.41,31.59,89.0,3.68,1.29,52.73,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,26.04,102808.0,24446.0,9582.0,0.07,31.20,9.32,43.43,77.0,2.55,0.75,38.17,XY24,NTT,MISC
5,APOLLOHOSP,6904.43,8285.00,-17.63,H-LC,80.21,112298.0,8732.0,11982.0,-0.13,8.43,10.67,20.00,30.0,0.73,0.82,23.69,X40N,BTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.2,-29.18,L-SC,98.96,142448.0,606.0,55968.0,-1.42,0.43,39.29,39.88,259.0,0.01,1.05,44.36,XY24,NTT,POWER


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,8.33,17237.0,-32356.0,1279.0,7.10,-65.24,7.42,-62.66,NaN,-25.30,0.13,15.60,XY24,ATH,MISC
28,DIGITIDE,188.38,237.72,NaN,NaN,3.12,45320.0,4065.0,6739.0,6.70,9.85,14.87,26.19,1.0,0.60,0.33,6.70,XY24,ATH,IT
63,QUESS,296.83,986.00,-30.58,M-SC,36.46,68766.0,3760.0,147166.0,4.61,5.78,214.01,232.18,191.0,0.03,0.50,5.30,XY24,NTT,MISC
33,GREENPANEL,375.16,537.00,202.97,M-SC,77.08,135347.0,-47731.0,126712.0,2.14,-26.07,93.62,43.14,216.0,-0.38,0.99,27.31,XY24,NTT,MISC
68,SAIL,130.64,228.00,95.22,M-MC,60.42,234278.0,9316.0,158349.0,2.03,4.14,67.59,74.53,185.0,0.06,1.72,36.02,XY24,BTT,STEEL


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,2347.98,4546.37,6.42,X-SC,100.00,99020.0,-1943.0,96475.0,-2.75,-1.92,97.43,93.63,64.0,-0.02,0.73,47.49,SR,ATH,CHEMICALS
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,68314.0,-45235.0,85290.0,-2.73,-39.84,124.85,35.27,261.0,-0.53,0.50,67.20,XR,NTT,HOTELS
54,KPIGREEN,497.21,732.16,27.10,H-SC,63.54,129931.0,4634.0,54571.0,-2.51,3.70,42.00,47.25,139.0,0.08,0.95,62.55,MH,ATH,POWER
69,SAMMAANCAP,170.35,326.00,-171.59,M-SC,87.50,82236.0,-19974.0,113362.0,-2.39,-19.54,137.85,91.37,206.0,-0.18,0.60,33.50,XY25,NTT,FINANCE
66,REPCOHOME,515.07,880.00,-53.80,H-SC,73.44,171578.0,-33420.0,178664.0,-2.06,-16.30,104.13,70.85,133.0,-0.19,1.26,38.51,XY24,NTT,FINANCE


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,54.03,M-SC,89.58,133596.0,-8883.0,9499.0,0.53,-6.23,7.11,0.44,243.0,-0.94,0.98,57.68,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-213.57,H-SC,97.92,81968.0,-13348.0,13385.0,-1.23,-14.00,16.33,0.04,154.0,-1.00,0.60,61.67,OX40N,NTT,MISC
79,SYMPHONY,1306.42,1306.0,-32.22,M-SC,5.21,144532.0,-26609.0,26551.0,-1.36,-15.55,18.37,-0.03,189.0,-1.00,1.06,3.88,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,124.33,H-SC,88.54,147225.0,-27334.0,27369.0,0.58,-15.66,18.59,0.02,127.0,-1.00,1.08,27.67,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-17.98,X-MC,16.67,86128.0,-16524.0,16519.0,-0.91,-16.10,19.18,-0.00,58.0,-1.00,0.63,30.92,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,3.12,45320.0,4065.0,6739.0,6.70,9.85,14.87,26.19,1.0,0.60,0.33,6.70,XY24,ATH,IT
83,TCS,3794.03,4998.00,-17.74,X-LC,42.71,267743.0,-28191.0,122091.0,0.08,-9.53,45.60,31.73,1.0,-0.23,1.97,7.17,X40,BTT,IT
48,INFY,1461.46,2275.00,-10.27,X-LC,59.38,288405.0,26804.0,118823.0,0.27,10.25,41.20,55.67,2.0,0.23,2.12,16.98,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-17.68,X-LC,20.83,257723.0,-12624.0,47241.0,0.24,-4.67,18.33,12.81,6.0,-0.27,1.89,7.78,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-11.84,X-LC,27.08,279513.0,14087.0,42821.0,-0.88,5.31,15.32,21.44,9.0,0.33,2.05,11.83,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,0.15,M-LC,39.58,175826.0,34986.0,9512.0,0.68,24.84,5.41,31.59,89.0,3.68,1.29,52.73,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.40,M-LC,26.04,102808.0,24446.0,9582.0,0.07,31.20,9.32,43.43,77.0,2.55,0.75,38.17,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,71.88,72880.0,38431.0,1516.0,-1.89,111.56,2.08,115.97,NaN,25.35,0.54,11.45,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,95.22,M-MC,60.42,234278.0,9316.0,158349.0,2.03,4.14,67.59,74.53,185.0,0.06,1.72,36.02,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,27.10,H-SC,63.54,129931.0,4634.0,54571.0,-2.51,3.70,42.00,47.25,139.0,0.08,0.95,62.55,MH,ATH,POWER
20,CAMS,3643.00,5250.99,5.87,H-SC,54.17,118891.0,16887.0,28141.0,0.07,16.56,23.67,44.14,124.0,0.60,0.87,35.49,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,26.04,102808.0,24446.0,9582.0,0.07,31.20,9.32,43.43,77.0,2.55,0.75,38.17,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,39.58,175826.0,34986.0,9512.0,0.68,24.84,5.41,31.59,89.0,3.68,1.29,52.73,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,71.88,72880.0,38431.0,1516.0,-1.89,111.56,2.08,115.97,NaN,25.35,0.54,11.45,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-37.67,M-SC,93.75,205462.0,33841.0,128229.0,1.33,19.72,62.41,94.43,198.0,0.26,1.51,51.64,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-3.77,M-LC,78.12,164703.0,13758.0,95692.0,0.42,9.11,58.10,72.51,69.0,0.14,1.21,13.45,XR,NTT,IT
45,INDIAMART,2327.09,4871.06,-47.70,H-SC,91.67,135961.0,12625.0,122202.0,-1.25,10.24,89.88,109.32,122.0,0.10,1.00,34.32,AR,ATH,MISC
54,KPIGREEN,497.21,732.16,27.10,H-SC,63.54,129931.0,4634.0,54571.0,-2.51,3.70,42.00,47.25,139.0,0.08,0.95,62.55,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,68314.0,-45235.0,85290.0,-2.73,-39.84,124.85,35.27,261.0,-0.53,0.50,67.20,XR,NTT,HOTELS
58,MASFIN,326.60,402.39,-18.25,H-SC,65.62,92700.0,-5280.0,28014.0,-0.19,-5.39,30.22,23.21,137.0,-0.19,0.68,34.45,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,6.42,X-SC,100.00,99020.0,-1943.0,96475.0,-2.75,-1.92,97.43,93.63,64.0,-0.02,0.73,47.49,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,26.04,102808.0,24446.0,9582.0,0.07,31.20,9.32,43.43,77.0,2.55,0.75,38.17,XY24,NTT,MISC
90,VALIANTORG,512.64,838.00,-353.90,H-SC,4.17,107900.0,-25386.0,109982.0,0.21,-19.05,101.93,63.47,148.0,-0.23,0.79,80.87,XR,NTT,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,ALKYLAMINE,2347.98,4546.37,6.42,X-SC,100.00,99020.0,-1943.0,96475.0,-2.75,-1.92,97.43,93.63,64.0,-0.02,0.73,47.49,SR,ATH,CHEMICALS
52,JPPOWER,18.73,26.20,-29.18,L-SC,98.96,142448.0,606.0,55968.0,-1.42,0.43,39.29,39.88,259.0,0.01,1.05,44.36,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-213.57,H-SC,97.92,81968.0,-13348.0,13385.0,-1.23,-14.00,16.33,0.04,154.0,-1.00,0.60,61.67,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,96.88,68314.0,-45235.0,85290.0,-2.73,-39.84,124.85,35.27,261.0,-0.53,0.50,67.20,XR,NTT,HOTELS
51,JIOFIN,310.58,387.00,6.53,H-LC,95.83,228340.0,10934.0,42563.0,-1.03,5.03,18.64,24.61,95.0,0.26,1.68,62.35,XY24,BTT,FINANCE


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.88
1,25,41.96
2,50,70.48


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.46
LC    34.03
MC    21.50
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.59
X40      12.24
XY25     12.00
X40N     11.81
XR       10.16
AR        8.22
OX40N     8.16
MH        1.70
X5K       1.49
X200      1.40
SR        1.22
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.64
M-SC    16.00
H-LC    14.14
X-LC    12.17
H-MC     9.51
X-MC     7.51
M-LC     6.67
X-SC     4.36
M-MC     4.12
L-SC     2.46
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.08,-1.76,35.79
IT,12.10,-10.41,64.63
BANKS,8.00,-4.60,47.28
FINANCE,7.88,-12.38,53.07
MISC,6.65,-14.73,67.96
PAINTS,5.69,-12.63,34.83
ELECTRICAL,5.46,0.79,45.55
HEALTHCARE,4.51,-4.62,35.18
AUTO,4.32,-14.64,65.04


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2699019.0,27
XR,1077244.0,13
AR,1025731.0,10
X40,653175.0,9
X40N,553584.0,11
XY25,482866.0,8
OX40N,319256.0,11
SR,178267.0,2
MH,83322.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1973664.0,20
M-SC,1764234.0,21
H-LC,614083.0,12
H-MC,538953.0,8
X-LC,516689.0,7
X-MC,428942.0,6
M-MC,409521.0,3
X-SC,325391.0,5
M-LC,263629.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      863606.0      6
M-SC       XY24      812408.0      7
H-SC       AR        523300.0      3
M-SC       XR        333725.0      4
M-MC       XY24      301867.0      2
H-MC       XY24      299340.0      3
X-LC       X40       240914.0      2
H-SC       X40N      224887.0      4
           XR        215571.0      3
H-LC       X40N      199269.0      4
M-SC       AR        182138.0      2
           OX40N     168751.0      5
H-LC       X40       157468.0      3
M-LC       XY24      154992.0      3
X-MC       X40       153427.0      2
L-SC       XR        147287.0      2
X-SC       XR        123817.0      1
X-LC       XY25      119445.0      3
           AR        117181.0      1
M-SC       XY25      113362.0      1
M-MC       XR        107654.0      1
X-MC       XY25      103045.0      1
           XY24       97415.0      1
X-SC       SR         96475.0      1
M-LC       XR         95692.0      1
H-SC       OX40N      91729.0      3
H-MC       XY25       87556.0      1
M-SC       SR         81792.0      1
H-LC       AR         79709.0      1
M-SC       X40        72058.0      1
H-LC       X200       69672.0      1
           X5K        65402.0      2
X-SC       XY24       62842.0      1
H-MC       AR         62255.0      1
L-SC       AR         59632.0      1
X-MC       X40N       58536.0      1
L-SC       XY24       55968.0      1
H-SC       MH         54571.0      1
L-MC       XR         53498.0      1
L-LC       XY25       46513.0      1
H-LC       XY24       42563.0      1
X-SC       OX40N      42257.0      2
X-LC       X40N       39149.0      1
H-MC       X40N       31743.0      1
           X40        29308.0      1
           MH         28751.0      1
X-MC       OX40N      16519.0      1
M-LC       XY25       12945.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
